# Sample aplikace pro interface

## pickle

`pickle` is a module in Python used for serializing and deserializing objects. Serialization, also known as pickling, involves converting a Python object into a byte stream, and deserialization, or unpickling, converts the byte stream back into an object.

In [ ]:
import pickle

# Example object
data = {'key': 'value', 'number': 42}

# Pickling the object
with open('data.pickle', 'wb') as file:
    pickle.dump(data, file)


In [ ]:
import pickle

# Unpickling the object
with open('data.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

print(loaded_data)


## Todolist

In [ ]:

import pickle


class Todolist:
    def __init__(self):
        self._todos = []
        self._finished = []

    def create_todo(self, what):
        self._todos.append(what)

    def last_todo(self):
        return self._todos[-1]

    def remove_todo(self, index):
        return self._todos.pop(index - 1)

    def __len__(self):
        return len(self._todos)

    def finish_todo(self, index):
        todo = self._todos.pop(index-1)
        self._finished.append(todo)

    def get_todos(self):
        return [t for t in enumerate(self._todos, start=1)]

    def get_finished(self):
        return [f for f in self._finished]

    def save_to_file(self, filename):
        with open(filename, "wb") as file:
            pickle.dump(self, file)

    @staticmethod
    def load_from_file(filename):
        with open(filename, "rb") as file:
            return pickle.load(file)



## CMD

```python
import cmd

from todolist import Todolist


class Todo(cmd.Cmd):
    intro = "This is a CLI for the todolist"
    prompt = "$ "

    def __init__(self):
        super().__init__()
        self.todolist = Todolist.load_from_file("todolist.pkl")

    def do_finish(self, arg):
        ids = tuple(map(int, arg.split()))
        for _id in ids:
            self.todolist.finish_todo(_id)

    def do_remove(self, arg):
        ids = tuple(map(int, arg.split()))
        for _id in ids:
            self.todolist.remove_todo(_id)

    def do_add(self, arg):
        self.todolist.create_todo(arg)

    def do_list(self, arg):
        print(arg)
        todos = self.todolist.get_todos()
        finished = self.todolist.get_finished()

        if len(todos) > 0:
            print("\nÚKOLY\n=====")
            for i, t in todos:
                print(f"{i}: {t}")
        else:
            print("Žádné úkoly ke splnění")

        if len(finished) > 0:
            print("\nHOTOVO\n======")
            for todo in finished:
                print(f"{todo}")
        print()

    def do_save(self, arg):
        self.todolist.save_to_file("todolist.pkl")

    def do_end(self, arg):
        print("Bye")
        return True


if __name__ == "__main__":
    Todo().cmdloop()

```

## TKinter

```python
import tkinter as tk

from todolist import Todolist


class SimpleApp:
    def __init__(self, root):
        self.root = root
        self.root.title("TODO list")

        self.todolist = Todolist()
        self.label_todo = tk.Label(root, text="TODO")
        self.label_todo.grid(row=0, column=0, padx=10, pady=10)

        self.label_finished = tk.Label(root, text="FINISHED")
        self.label_finished.grid(row=0, column=2, padx=10, pady=10)

        # Create a Listbox in the first column
        self.listbox = tk.Listbox(root)
        self.listbox.grid(row=1, column=0, rowspan=4, padx=10, pady=10)

        self.finished = tk.Listbox(root)
        self.finished.grid(row=1, column=2, rowspan=4, padx=10, pady=10)

        # Create a Text Entry Field in the second column
        self.entry = tk.Entry(root)
        self.entry.grid(row=1, column=1, padx=10, pady=10)

        # Create an Add Button in the second column
        self.add_button = tk.Button(root, text="Add", command=self.add_item)
        self.add_button.grid(row=2, column=1, padx=10, pady=10)
        self.finish_button = tk.Button(root, text="Finish", command=self.finish_item)
        self.finish_button.grid(row=3, column=1, padx=10, pady=10)

        # Create a Delete Button in the second column
        self.delete_button = tk.Button(root, text="Delete", command=self.delete_item)
        self.delete_button.grid(row=4, column=1, padx=10, pady=10)

    def add_item(self):
        todo = self.entry.get()
        self.entry.delete(0, tk.END)
        self.todolist.create_todo(todo)
        self.update_listboxes()

    def update_listboxes(self):
        self.listbox.delete(0, tk.END)
        for _, t in self.todolist.get_todos():
            self.listbox.insert(tk.END, t)

        self.finished.delete(0, tk.END)
        for t in self.todolist.get_finished():
            self.finished.insert(tk.END, t)

    def finish_item(self):
        try:
            selected_index = self.listbox.curselection()[0]
            self.todolist.finish_todo(selected_index + 1)
            self.update_listboxes()
        except IndexError:
            pass

    def delete_item(self):
        try:
            selected_index = self.listbox.curselection()[0]
            self.todolist.remove_todo(selected_index + 1)
            self.update_listboxes()
        except IndexError:
            pass


def main():
    root = tk.Tk()
    app = SimpleApp(root)
    root.mainloop()


if __name__ == "__main__":
    main()

```